In [1]:
import torch
import gurobipy as gp

In [2]:
from learn.train import build_inst, build_graphs, remove_redundant_nodes, get_train_mask, get_solution_mask, get_mask_node_feature
from learn.model import Model, FocalLoss
from learn.generator import maximum_independent_set_problem
from learn.info import ModelInfo
from learn import solver

In [3]:
import json

def with_lic(m):
    with open("gb.lic") as f:
        env = gp.Env(params=json.load(f))
    return m.copy(env=env)
        

In [4]:
# m = maximum_independent_set_problem(num_nodes=2000)
m = gp.read("model_11.lp")
info = ModelInfo.from_model(m)
m = with_lic(m)
m.setParam("MIPFocus", 1)
m.setParam("RINS", 10)
m.setParam("TimeLimit", 60)
m.setParam("NoRelHeurTime", 128)
m.optimize()
# m.dispose()
# x = [v.X for v in m.getVars()]

Restricted license - for non-production use only - expires 2026-11-23
Read LP format model from file model_11.lp
Reading time = 0.68 seconds
: 330539 rows, 111228 columns, 999160 nonzeros
Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "831775-C3Dev"
Set parameter CSAppName to value "Josh"
Compute Server job ID: 58a0c24f-89f3-45db-96ff-8bf5fd2bc88d
Capacity available on '831775-C3Dev' cloud pool - connecting...
Established HTTPS encrypted connection
Set parameter MIPFocus to value 1
Set parameter RINS to value 10
Set parameter TimeLimit to value 60
Set parameter NoRelHeurTime to value 128
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.4.0 22E252)
Gurobi Compute Server Worker version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8375C CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-de

In [5]:
len(m.getVars())

111228

In [6]:
edges = [[], []]
for con_i, lhs_p in enumerate(info.con_info.lhs_p):
    shifted_con_i = con_i + info.var_info.n
    for var_i in lhs_p: 
        edges[0].append(var_i)
        edges[1].append(shifted_con_i)

In [7]:
parts = solver.fennel_partition(edges, 4, 0.3, 1)
sub_mappings = []
sub_infos = []
for var_idxs, _ in parts:
    sub_info, sub_mapping = info.subset(var_idxs)
    sub_infos.append(sub_info)
    sub_mappings.append(sub_mapping)

27520
63232
------------------------------------------------------------------------------
27535
62053
------------------------------------------------------------------------------
27474
63168
------------------------------------------------------------------------------
27516
62981
------------------------------------------------------------------------------


In [8]:
stitch_x = [0 for _ in range(info.var_info.n)]
for sub_info, sub_mapping in zip(sub_infos, sub_mappings):
    cur_m, _ = solver.build_partial_model(sub_info)
    cur_m = with_lic(cur_m)
    cur_m.optimize()
    cur_x = [v.x for v in cur_m.getVars()]
    cur_m.dispose()
    for new_i, old_i in sub_mapping.items():
        stitch_x[old_i] = cur_x[new_i]

Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "831775-C3Dev"
Set parameter CSAppName to value "Josh"
Compute Server job ID: 6eb0540d-7119-4df7-8d66-d8d5891661ae
Capacity available on '831775-C3Dev' cloud pool - connecting...
Established HTTPS encrypted connection
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.4.0 22E252)
Gurobi Compute Server Worker version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8375C CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
CSIdleTimeout  1800

Optimize a model with 154943 rows, 63232 columns and 404355 nonzeros
Model fingerprint: 0x1fc4a5e0
Variable types: 21590 continuous, 41642 integer (41298 binary)
Coefficient statistics:
  Matrix range     [6e-02, 1e+03]
  Objective range  [7e-05, 5e-01]
  Bounds range     [1e+00, 6e+02]
  R

In [9]:
import numpy as np
import random

In [72]:
import random
fixed = list([i for i in range(info.var_info.n) if info.var_info.types[i] == "B"])
random.shuffle(fixed)
fixed = fixed[:int(len(fixed) * 0.05)]

In [73]:
# def unfix_vars_for_feasibility(model, bvars, bounds):
#     model.feasRelax(0, False, bvars, [1 for _ in bvars], [1 for _ in bvars], [], [])
#     model.setParam("MIPFocus", 1)
#     model.setParam("SolutionLimit", 1)
#     model.setParam("NoRelHeurTime", 128)
#     model.optimize()
#     return [v.x != b for v, b in zip(bvars, bounds)]

In [74]:
# lhs, var_vals, rhs = solver.get_constraint_side_matrices(stitch_x, info.con_info)
# ops = np.array(info.con_info.types)[:, np.newaxis]
# violations = solver.get_constraint_violations(lhs, var_vals, rhs, ops)
# fixed = solver.unfix_violation_variables(lhs, var_vals, rhs, ops, info.var_info, info.con_info, violations, to_keep)

In [75]:
new_model, new_mapping = solver.build_partial_model(info, const_vars={i: stitch_x[i] for i in fixed})
new_model = with_lic(new_model)

Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "831775-C3Dev"
Set parameter CSAppName to value "Josh"
Compute Server job ID: fd8d7402-c2b7-4f72-b6a9-221372e25b97
Capacity available on '831775-C3Dev' cloud pool - connecting...
Established HTTPS encrypted connection


In [66]:
# all_vars = new_model.getVars()
# bvars = [all_vars[i] for i in fixed]
# bounds = [stitch_x[i] for i in fixed]
# need_relax = unfix_vars_for_feasibility(new_model, bvars, bounds)

In [67]:
# all_vars = new_model.getVars()
# bvars = [all_vars[i] for i in fixed]
# bounds = [stitch_x[i] for i in fixed]
# need_relax = unfix_vars_for_feasibility(new_model, bvars, bounds)

In [68]:
# print(len(fixed))
# f = list(fixed)
# for i in range(len(f)):
#     if need_relax[i]:
#         fixed.remove(f[i])
# print(len(fixed))    

In [69]:
# new_model, new_mapping = solver.build_partial_model(info, const_vars={i: stitch_x[i] for i in fixed})

In [76]:
# new_model = with_lic(new_model)
# new_model.setParam("MIPFocus", 1)
# new_model.setParam("NoRelHeurTime", 128)
# new_model.optimize()

In [77]:
for i in range(1000):
    print(i)
    print("^"*100)
    cnds = get_iis_candidates(new_model)

    counter = 0
    for v in new_model.getVars():
        if v.var_name in cnds and v.index in fixed:
            v.lb = 0
            v.ub = 1
            counter += 1
    print(counter)

    if not cnds:
        print("YES!!!!")
        break
    # unfix = set()
    # for c in new_model.getConstrs():
    #     if c.IISConstr:
    #         unfix.update(info.con_info.lhs_p[c.index])
    # for v in new_model.getVars():
    #     if v.IISLB or v.IISLB:
    #         unfix.add(v.index)
    
    # print(len(fixed))
    # for i in unfix:
    #     if i in fixed:
    #         fixed.remove(i)
    # print(len(fixed))

    # new_model.dispose()
    # new_model, new_mapping = solver.build_partial_model(info, const_vars={i: stitch_x[i] for i in fixed})
    # new_model = with_lic(new_model)
    # new_model.optimize()
    

0
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.4.0 22E252)
Gurobi Compute Server Worker version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8375C CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
CSIdleTimeout  1800

IIS computation: initial model status unknown, solving to determine model status

IIS computed: 1 constraints, 2 bounds
IIS runtime: 0.00 seconds (0.00 work units)
13
1
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.4.0 22E252)
Gurobi Compute Server Worker version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8375C CPU

In [82]:
sum(v.lb == v.ub for v in new_model.getVars())

AttributeError: Failure writing output to destination, passed 74 returned 0 (code 23, node https://ip-10-0-77-164:61000, command PUT https://compute-us-east-1.gurobi.com/api/v1/jobs/fd8d7402-c2b7-4f72-b6a9-221372e25b97/worker?sync=true&cmd=14)

In [49]:
new_model = with_lic(new_model)

Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "831775-C3Dev"
Set parameter CSAppName to value "Josh"
Compute Server job ID: 9da52c94-a2b3-416f-9a87-de5b2c20049a
Capacity available on '831775-C3Dev' cloud pool - connecting...
Established HTTPS encrypted connection


GurobiError: Unable to copy model out of Compute Server

In [51]:

new_model.computeIIS()
# new_model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.4.0 22E252)
Gurobi Compute Server Worker version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8375C CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
CSIdleTimeout  1800


IIS computed: 1 constraints, 2 bounds
IIS runtime: 0.00 seconds (0.00 work units)


In [52]:
new_model.write("iis.ilp")

In [53]:
unfix = set()
for c in new_model.getConstrs():
    if c.IISConstr:
        unfix.update(info.con_info.lhs_p[c.index])
for v in new_model.getVars():
    if v.IISLB or v.IISLB:
        unfix.add(v.index)

In [61]:
def get_iis_candidates(model) -> set:
    try:
        model.computeIIS()
    except Exception as e:
        print(e)
        return set()
        
    model.write("temp_iis.ilp")
    with open("temp_iis.ilp") as f:
        candidates = set(f.read().split())
    return candidates
        

In [59]:
for v in new_model.getVars():
    if v.index in unfix:
        print(v.var_name)

C58721
C58723
C58756
C58782
C58808
C58834
C58860
C58886
C58912
C58938
C58964
C58990
C59016
C59042


In [56]:
with open("iis.ilp") as f:
    parts = f.read().split()
    print(set(parts))

{'C63233', 'LP', 'detail.', 'C63062', 'C62932', 'C63088', 'C63114', 'C63140', 'C63166', 'full', 'MPS', 'capture', 'Subject', 'C62984', 'Signature:', '\\', 'model', '_copy_copy', 'format', 'Model', 'Use', 'C63036', 'to', 'C63218', 'Maximize', 'for', '-13', 'browsing.', '=', 'R252947:', '-', '<=', 'C63192', 'To', 'C62958', 'C63010', 'Bounds', '0x6e6bbe1c0f2f3b80', 'Binaries', 'End', '0', 'C62906'}


In [ ]:
assert 1 == 2

In [ ]:
for i in range(100):
    new_model, new_mapping = solver.build_partial_model(info, const_vars={i: stitch_x[i] for i in fixed})
    new_model = with_lic(new_model)
    try:
        
        new_model.computeIIS()
    except:
        print("LETS GOOOOO!!!!")
        print("^"*78)
        break
    
    to_remove = set()
    for i, v in enumerate(new_model.getVars()):
        if v.IISLB or v.IISLB:
            to_remove.add(i)
        
    for r in to_remove:
        if r in fixed:
            fixed.remove(r)
    new_model.dispose()
    print(len(fixed))
    print('-'*100)
    

new_model.optimize()
final_x = [x.x for x in new_model.getVars()]
final_final_x = stitch_x.copy()

for new_i, old_i in new_mapping.items():
    final_final_x[old_i] = final_x[new_i]

In [ ]:
new_model.write("iis.ilp")

In [ ]:
assert 1 == 2

In [ ]:
lhs, var_vals, rhs = solver.get_constraint_side_matrices(final_x, info.con_info)
ops = np.array(info.con_info.types)[:, np.newaxis]
violations = solver.get_constraint_violations(lhs, var_vals, rhs, ops)

In [ ]:
%%capture

inst = build_inst(maximum_independent_set_problem, 10)
graphs = build_graphs(inst)
for g in graphs:
    remove_redundant_nodes(g)

In [ ]:
mask_feat_size = 2
n_node_feats = graphs[0].ndata['feat'].shape[1] + mask_feat_size
n_edge_feats = graphs[0].edata['feat'].shape[1]
num_classes = int(graphs[0].ndata['label'].max()) + 1
hidden_size = 64

In [ ]:
model = Model(n_node_feats, n_edge_feats, hidden_size, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [ ]:
import random

print("Total number of graphs", len(graphs))
num_epochs = 500
for epoch in range(num_epochs):
    
    model.train()
    optimizer.zero_grad()
    
    cntr = 0
    loss = 0

    random.shuffle(graphs)
    for i, g in enumerate(graphs):

        train_mask = get_train_mask(g, ratio=0.8)
        solution_mask = get_solution_mask(train_mask, (0.2, 1.0))
        node_feat_with_hint, hint_mask = get_mask_node_feature(g.ndata['feat'], g.ndata['label'], solution_mask)
        
        logits = model(g, node_feat_with_hint, g.edata['feat'])
        labels = g.ndata['label']

        n_vars = g.ndata['feat'][:, 2].sum().int()
        loss += FocalLoss()(
            logits[:n_vars][~hint_mask[:n_vars]], 
            labels[:n_vars][~hint_mask[:n_vars]]
        )
        cntr += 1

        if cntr == 256:
            print("loss", loss.detach().numpy())
            print("#"*78)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()

            loss = 0
            cntr = 0

    print("-"*78)
    print(logits[:n_vars][~hint_mask[:n_vars]].detach().numpy())
    print(labels[:n_vars][~hint_mask[:n_vars]].detach().numpy())
    print('^'*78)

In [ ]:
# TODO: normalize violation by constraint type
import numpy as np
from typing import Set, List
from learn.info import ConInfo


def get_constraint_side_matrices(x, con_info):
    
    n_con = len(con_info.lhs_p)
    n_var = len(solution)
    lhs = np.zeros((n_con, n_var))
    
    for con_idx in range(n_con):
        var_idxs = con_info.lhs_p[con_idx]
        var_cefs = con_info.lhs_c[con_idx]
        for var_idx, var_cef in zip(var_idxs, var_cefs):
            lhs[con_idx][var_idx] = var_cef

    var_vals = np.array(solution)[:, np.newaxis]
    rhs = np.array(con_info.rhs)[:, np.newaxis]
    return lhs, var_vals, rhs

# lhs, vs, rhs = get_constraint_side_matrices(x, con_info)
# ops = np.array(con_info.types)[:, np.newaxis]


def get_constraint_violations(lhs, vs, rhs, ops):
    lt_ops = ops == ConInfo.ENUM_TO_OP["<="]
    eq_ops = ops == ConInfo.ENUM_TO_OP["=="]
    gt_ops = ops == ConInfo.ENUM_TO_OP[">="]
    
    diff = rhs - lhs @ vs
    violations = np.zeros_like(diff, dtype=bool)
    violations[lt_ops] = diff[lt_ops] > 0
    violations[gt_ops] = diff[gt_ops] < 0
    violations[eq_ops] = diff[eq_ops] != 0
    diff[violations] = 0
    return np.abs(diff)
    

def get_high_confidence_prediction_indices(ps, ratio):
    # TODO: replace with top_k
    n_remain = int(len(ps) * ratio)
    return sorted(range(len(ps)), key=lambda i: ps[i], reverse=True)[:n_remain]


# TODO: use rust
def unfix_violation_variables(lhs, x, rhs, ops, var_info, con_info, violations, fixed: List[int], ratio):
    fixed = set(fixed)
    
    violations = get_constraint_violations(lhs, x, rhs, ops)
    violations_idxs = np.where(violations != 0)[0]
    violations_idxs = sorted(violations_idxs, key=lambda idx: violations[idx])
    check_n_consts = min(int(len(violations) * ratio), len(violations_idxs))
    ops = ops.reshape(-1)
    
    for i in range(check_n_consts):
        diff = violations[i]
        lhs_p = con_info.lhs_p[i]
        lhs_c = con_info.lhs_c[i]
        op = ops[i]
        p_c = sorted(zip(lhs_p, lhs_c), key=lambda pc: pc[0] in fixed)
        for p, c in p_c:
            
            if diff <= 0:
                break
            
            if con_info.OP_TO_ENUM[op] == ">=":
                
                if c >= 0:
                    diff -= (var_info.ubs[p] - x[p]) * c
                else:
                    diff -= (var_info.lbs[p] - x[p]) * c

            if con_info.OP_TO_ENUM[op] == "<=":
                if c >= 0:
                    diff += (var_info.lbs[p] - x[p]) * c
                else:
                    diff += (var_info.ubs[p] - x[p]) * c
            
            if p in fixed:
                fixed.remove(p)
            
    return fixed

In [ ]:
lhs, var_vals, rhs = get_constraint_side_matrices(solution, info.con_info)
ops = np.array(info.con_info.types)[:, np.newaxis]
violations = get_constraint_violations(lhs, var_vals, rhs, ops)

In [ ]:
unfix_violation_variables(lhs, var_vals, rhs, ops, info.var_info, info.con_info, violations, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ratio=0.4)

In [ ]:
from learn.info import ModelInfo
m = maximum_independent_set_problem()
info = ModelInfo.from_model(m)
m.optimize()
solution = [v.X for v in m.getVars()]
for i in range(5):
    solution[i] = 1

In [ ]:
# def deep_solve(problem, solution=None):
#     sub_problems = partition(problem)    
#     for p in sub_problems:
        
#         if has_solution(p):
#             seed = solution[p]
#             break
            
#         if is_small(p):
#             seed = exact_solve(p)
#             break

#         seed = deep_solve(problem)
    
#     infer(seed)
#     solution = recursive_cross()
#     return solution
    

    
        
        
    
    
        
    
    
        
        

In [ ]:
a = np.array([1, 0, 2]).sort(key=1)

In [ ]:
ps = [0, 0, 1]
n_remain = int(len(ps) * 0.7)
sorted(range(len(ps)), key=lambda i: ps[i])[:n_remain]

In [ ]:
solution[0] = 1
solution[2] = 1
solution[3] = 1
solution[4] = 1
solution[5] = 1
solution[6] = 1
solution[7] = 1

In [ ]:
solution = [1 for _ in range(len(solution))]

In [ ]:
get_constraint_violations(solution, info.con_info)

In [ ]:
assert 1 == 2